<p style="font-weight:bold;"> <span style="font-size: 36px"> Import Calculation Methods </span> </p

In [ ]:
#!import "./Extensions"

# Get Previous Identities

In [ ]:
public static Dictionary<AocStep,IEnumerable<AocStep>> GetPreviousIdentities(IEnumerable<AocStep> identities)
{
    var bopNovelties = identities.Where(id => id.AocType == AocTypes.BOP).Select(id => id.Novelty);
    var previousStep = (new string[]{Novelties.N,Novelties.I,Novelties.C})
                                        .ToDictionary(n => n, n => bopNovelties.Contains(n) ? new AocStep(AocTypes.BOP,n) : null);
    return identities.Where(id =>  id.AocType != AocTypes.BOP)
                                .ToDictionary(x => x, x => {var ret = x.AocType == AocTypes.CL ? previousStep.Where(kvp => kvp.Value != null).Select(kvp => kvp.Value).ToArray() 
                                                                                               : previousStep[x.Novelty].RepeatOnce();
                                                            previousStep[x.Novelty] = new AocStep(x.AocType, x.Novelty);
                                                            return ret;});
}

# Get Reference AocStep for calculated steps

In [ ]:
public static AocStep GetReferenceAocStepForCalculated(this IEnumerable<AocStep> identities, Dictionary<AocStep, AocConfiguration> aocConfigurationByAocStep, AocStep identityAocStep)
{
    return identities.LastOrDefault(aocStep => aocConfigurationByAocStep[aocStep].DataType != DataType.Calculated
                                            && aocConfigurationByAocStep[aocStep].DataType != DataType.CalculatedTelescopic
                                            && aocConfigurationByAocStep[aocStep].Order < aocConfigurationByAocStep[identityAocStep].Order
                                            && aocStep.Novelty == identityAocStep.Novelty) 
        ?? new AocStep(default, default);
}

# Discount and Cumulate

In [ ]:
public static double[] ComputeDiscountAndCumulate(this double[] nominalValues, double[] monthlyDiscounting, PeriodType periodType) 
{ 
    if(nominalValues == null) return Enumerable.Empty<double>().ToArray();
    
    var ret = new double[nominalValues.Length];
    
    if(periodType == PeriodType.BeginningOfPeriod)
    {
        for (var i = nominalValues.Length - 1; i >= 0; i--)
                ret[i] = nominalValues[i] + GetElementOrDefault(ret, i + 1) * GetElementOrDefault(monthlyDiscounting, i/12);
        return ret;
    }
    
    for (var i = nominalValues.Length - 1; i >= 0; i--)
                ret[i] = ( nominalValues[i] + GetElementOrDefault(ret, i + 1) ) * GetElementOrDefault(monthlyDiscounting, i/12);
    
    return ret;
}

In [ ]:
public static double[] ComputeDiscountAndCumulateWithCreditDefaultRisk(this double[] nominalValues, double[] monthlyDiscounting, double nonPerformanceRiskRate) //Is it correct that NonPerformanceRiskRate is a double? Should it be an array that takes as input tau/t?
{ 
    return Enumerable.Range(0, nominalValues.Length)
                     .Select( t => Enumerable.Range(t, nominalValues.Length-t)
                                             .Select( tau => nominalValues[tau] * Math.Pow(GetElementOrDefault(monthlyDiscounting, t/12), tau-t+1) * (Math.Exp(-nonPerformanceRiskRate*(tau-t)) - 1) )
                                             .Sum() )
                     .ToArray();
}

# Import Configuration 

## Data Nodes

In [ ]:
public GroupOfInsuranceContract ExtendGroupOfContract(GroupOfInsuranceContract gic, IDataRow datarow) => gic;

In [ ]:
public GroupOfReinsuranceContract ExtendGroupOfContract(GroupOfReinsuranceContract gric, IDataRow datarow) => gric;

## Change Sign Rules

In [ ]:
public static int GetSign((string AocType, string AmountType, string EstimateType, bool IsReinsurance) variable, Systemorph.Vertex.Hierarchies.IHierarchicalDimensionCache hierarchyCache) => 1;

## EstimateType And AmountType

In [ ]:
public static Dictionary<string, HashSet<string>> GetAmountTypesByEstimateType(Systemorph.Vertex.Hierarchies.IHierarchicalDimensionCache hierarchyCache)
{
    return new Dictionary<string, HashSet<string>>(){
        {EstimateTypes.RA, new string[]{}.ToHashSet()},
        {EstimateTypes.C, new string[]{}.ToHashSet()},
        {EstimateTypes.L, new string[]{}.ToHashSet()},
        {EstimateTypes.LR, new string[]{}.ToHashSet()},
   };
}

## Technical Margin EstimateType

In [ ]:
public static HashSet<string> GetTechnicalMarginEstimateType()
{
    return new []{EstimateTypes.C, EstimateTypes.L, EstimateTypes.LR, }.ToHashSet();
}